<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="SN_web_lightmode.png logo"/>
</center>

# Investigation of diabetes patients readmission among US hospitals


# Lab 2. Data Wrangling

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Handle missing values
*   Correct data format
*   Standardize and normalize data

<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="https://#identify_handle_missing_values">Identify and handle missing values</a>
        <ul>
            <li><a href="https://#identify_missing_values">Identify missing values</a></li>
            <li><a href="https://#deal_missing_values">Deal with missing values</a></li>
            <li><a href="https://#correct_data_format">Correct data format</a></li>
        </ul>
    </li>
    <li><a href="https://#data_standardization">Data standardization</a></li>
    <li><a href="https://#data_normalization">Data normalization (centering/scaling)</a></li>
    <li><a href="https://#binning">Binning</a></li>
    <li><a href="https://#indicator">Indicator variable</a></li>
</ul>

</div>

<hr>


<h2>What is the purpose of data wrangling?</h2>


Data wrangling is the process of converting data from the initial format to a format that may be better for analysis.


<h3>How to change "Yes" and "No" to True and False?</h3>


<h3>Import data</h3>
<p>
You can find the "Diabetes" from the following link: <a href="https://www.kaggle.com/datasets/brandao/diabetes">https://www.kaggle.com/datasets/brandao/diabetes</a>.
We will be using this dataset throughout this course.
</p>


<h4>Import needed libraries</h4> 


In [ ]:
conda install -c anaconda scikit-learn

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from matplotlib import pyplot
import pandas as pd
import sklearn as sklearn
import scipy.stats as stats
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_percentage_error
import math
import numpy as np

<h2>Reading the dataset from the URL and adding the related headers</h2>


First, we assign the URL of the dataset to "filename".


In [ ]:
filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX09J0EN/diabetes.csv"

Then, we create a Python list <b>headers</b> containing name of headers.


Use the Pandas method <b>read_csv()</b> to load the data from the web address. Set "df.columns" equal to the Python list "headers".


In [ ]:
df = pd.read_csv(filename)

In [ ]:
headers = ["Encounter Id","Patient Number","Race","Gender","Age","Weight","Admission Type Id",
           "Discharge Disposition Id","Admission Source Id","Time In Hospital","Payer Code",
           "Medical Specialty","Num Lab Procedures","Num Procedures","Num Medications",
           "Number Outpatient","Number Emergency","Number Inpatient","Diagnosis1","Diagnosis2",
           "Diagnosis3","Number Diagnoses","Max Glu Serum","A1c Result","Metformin","Repaglinide",
           "Nateglinide","Chlorpropamide","Glimepiride","Acetohexamide","Glipizide",
           "Glyburide","Tolbutamide","Pioglitazone","Rosiglitazone","Acarbose","Miglitol",
           "Troglitazone","Tolazamide","Examide","Citoglipton","Insulin","Glyburide-metformin",
           "Glipizide-metformin","Glimepiride-pioglitazone","Metformin-rosiglitazone","Metformin-pioglitazone",
           "Change","Diabetes Medication","Readmitted"]
df.columns = headers

Use the method <b>head()</b> to display the first five rows of the dataframe.


In [ ]:
# To see what the data set looks like, we'll use the head() method.
df.head()

As we can see, several question marks appeared in the dataframe; those are missing values which may hinder our further analysis.

<div>So, how do we identify all those missing values and deal with them?</div> 

<b>How to work with missing data?</b>

Steps for working with missing data:

<ol>
    <li>Identify missing data</li>
    <li>Deal with missing data</li>
    <li>Correct data format</li>
</ol>


<h2 id="identify_handle_missing_values">Identify and handle missing values</h2>

<h3 id="identify_missing_values">Identify missing values</h3>
<h4>Convert "?" to NaN</h4>
In the diabetes dataset, missing data comes with the question mark "?".
We replace "?" with NaN (Not a Number), Python's default missing value marker for reasons of computational speed and convenience. Here we use the function: 
 <pre>.replace(A, B, inplace = True) </pre>
to replace A by B.


In [ ]:
df.replace("?", np.nan, inplace = True)
df.head(5)

<h4>Evaluating for Missing Data</h4>

The missing values are converted by default. We use the following functions to identify these missing values. There are two methods to detect missing data:
<ol>
    <li><b>.isnull()</b></li>
    <li><b>.notnull()</b></li>
</ol>
The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.


In [ ]:
missing_data = df.isnull()
missing_data.head(5)

"True" means the value is a missing value while "False" means the value is not a missing value.


<h4>Count missing values in each column</h4>
<p>
Using a for loop in Python, we can quickly figure out the number of missing values in each column. As mentioned above, "True" represents a missing value and "False" means the value is present in the dataset.  In the body of the for loop the method ".value_counts()" counts the number of "True" values. 
</p>


In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Based on the summary above, each column has 101766 rows of data and seven of the columns containing missing data:

<ol>
    <li>"Race": 2273 missing data</li>
    <li>"Weight": 98569 missing data</li>
    <li>"PayerCode": 40256 missing data</li>
    <li>"MedicalSpecialty" : 49949 missing data</li>
    <li>"Diagnosis1": 21 missing data</li>
    <li>"Diagnosis2": 358 missing data</li>
    <li>"Diagnosis3": 1423 missing data</li>
</ol>


<h3 id="deal_missing_values">Deal with missing data</h3>
<b>How to deal with missing data?</b>

<ol>
    <li>Drop data<br>
        a. Drop the whole row<br>
        b. Drop the whole column
    </li>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>


Whole columns should be dropped only if most entries in the column are empty. In our dataset, none of the columns are empty enough to drop entirely.
We have some freedom in choosing which method to replace data; however, some methods may seem more reasonable than others. We will apply some of these methods to different columns:

<b>Replace by frequency:</b>
<ul>
    <li>"Race": 2273 missing data
        <ul>
            <li>Reason: "Race" column cannot be replaced by mean value, so the obvious decision would be to find the most frequent value and replace Nan data.</li>
        </ul>
    </li>
    <li>"PayerCode": 40256 missing data
        <ul>
            <li>Reason: "PayerCode" column cannot be replaced by mean value, and 40256 rows are too much to simply throw away. So just replace empty rows with the most frequent value.</li>
        </ul>
    </li>
    <li>"MedicalSpecialty" : 49949 missing data</li>
     <ul>
            <li>Reason: "MedicalSpecialty" column cannot be replaced by mean value, and thrown away. So replace them with the most frequent value.</li>
        </ul>
    <li>"Diagnosis2": 358 missing data, replace it with the most frequent value.</li>
    <li>"Diagnosis3": 1423 missing data, replace it with the most frequent value.</li>
    <li>"Weight": 98569 missing data, replace it with the most frequent value.</li>
</ul>
<b>Drop the whole row:</b>
<ul>
    <li>"Diagnosis1": 21 missing rows is not much, so simply delete them.</li>
</ul>

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1: </h1>

<b>Based on the example above, replace NaN in "PayerCode" column with the most frequent value.</b>

</div>

In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
#Calculate the most frequent vaule for "PayerCode" column
most_frequent_payer_code = df["Payer Code"].value_counts().idxmax()
print("Average payer code:", most_frequent_payer_code)

# Replace "NaN" with the mean value in the "PayerCode" column</h4>

df['Payer Code'].replace(np.nan, most_frequent_payer_code, inplace=True)
```
</details>

<h4>Calculating the mean value for column</h4>

<h4>Replace "NaN" with the mean value in the column</h4>

<h4>Calculate the most frequent value for the "Race",Diagnosis2","Diagnosis3","Weight" column</h4>

In [ ]:
avg_race = df['Race'].value_counts().idxmax()
avg_diagnosis_2 = df['Diagnosis2'].value_counts().idxmax()
avg_diagnosis_3 = df['Diagnosis3'].value_counts().idxmax()
avg_weight = df['Weight'].value_counts().idxmax()

print("Average Race:", avg_race)
print("Average second diagnosis:", avg_diagnosis_2)
print("Average third diagnosis:", avg_diagnosis_3)
print("Average weight:", avg_weight)

<h4>Replace "NaN" with the mean value in the "Race",Diagnosis2","Diagnosis3","Weight" column</h4>


In [ ]:
df['Race'].replace(np.nan, avg_race, inplace=True)
df['Diagnosis2'].replace(np.nan, avg_diagnosis_2, inplace=True)
df['Diagnosis3'].replace(np.nan, avg_diagnosis_3, inplace=True)
df['Weight'].replace(np.nan, avg_weight, inplace=True)

<h4>Calculate the most frequent value for "PayerCode" column</h4>


In [ ]:
most_frequent_payer_code = df["Payer Code"].value_counts().idxmax()
print("Most frequent payer code:", most_frequent_payer_code)
print(df["Payer Code"].value_counts())

<h4>Replace "NaN" with the mean value in the "PayerCode" column</h4>


In [ ]:
df['Payer Code'].replace(np.nan, most_frequent_payer_code, inplace=True)

To see which values are present in a particular column, we can use the ".value_counts()" method:


In [ ]:
df['Medical Specialty'].value_counts()

We can see that InternalMedicine are the most common type. We can also use the ".idxmax()" method to calculate the most common type automatically:


In [ ]:
df['Medical Specialty'].value_counts().idxmax()

The replacement procedure is very similar to what we have seen previously:


In [ ]:
#replace the missing "MedicalSpecialty' values by the most frequent
df["Medical Specialty"].replace(np.nan, "InternalMedicine", inplace=True)

Finally, let's drop all rows that do not have "Diagnosis1" data:

In [ ]:
# simply drop whole row with NaN in "Diagnosis1" column
df.dropna(subset=["Diagnosis1"], axis=0, inplace=True)

# reset index, because we droped two rows
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

<b>Good!</b> Now, we have a dataset with no missing values.


<h2>Analyze data loss among restored dataframe.</h2>

Here we will investigate how much data we will lose after replacing it by mean and the most frequent value.

In [ ]:
#this code creates new dataframes df2 df3_clean removes all nan values from columns
df2 = pd.read_csv(filename)
df2.columns = headers
df2.replace("?", np.nan, inplace = True)
#here we drops nan data in mentioned columns
df2.dropna(subset=["Race"], axis=0, inplace=True)
df2.dropna(subset=["Time In Hospital"], axis=0, inplace=True)
df2.dropna(subset=["Gender"], axis=0, inplace=True)
df2.dropna(subset=["Num Lab Procedures"], axis=0, inplace=True)
df2.dropna(subset=["Diabetes Medication"], axis=0, inplace=True)
df3_clean = df2.copy()

#this part generates nan values among columns
l = [0.15, 0.1, 0.12, 0.1, 0.01]
col = ['Race', 'Time In Hospital', 'Gender', 'Num Lab Procedures', 'Diabetes Med']
for i in range(len(l)):
    m = np.random.rand(len(df2))
    mask = m < l[i]
    df2.loc[mask, col[i]] = np.NaN


After adding nan values we need to restore columns by different methods.

In [ ]:
# replacing nan values by the most frequent value
avg_race = df2['Race'].value_counts().idxmax()
df2['Race'].replace(np.nan, avg_race, inplace=True)

avg_gender = df2['Gender'].value_counts().idxmax()
df2['Gender'].replace(np.nan, avg_gender, inplace=True)

diabetes_meds = df2['Diabetes Medication'].value_counts().idxmax()
df2['Diabetes Medication'].replace(np.nan, diabetes_meds, inplace=True)

# replacing nan values mean value
lab_procedures = df2['Num Lab Procedures'].astype('float').mean(axis=0)
df2['Num Lab Procedures'].replace(np.nan, lab_procedures, inplace=True)

avg_time_in_hospital = df2['Time In Hospital'].astype('float').mean(axis=0)
df2['Time In Hospital'].replace(np.nan, avg_time_in_hospital, inplace=True)

To find error we will use sklearn.metrics, .mean_absolute_percentage_error for float variables and .recall_score for object values.

In [ ]:
cm = recall_score(df3_clean['Race'],df2['Race'],average='micro')
print("Error of Race :", math.trunc((1 - cm)*100),'%')

cm = recall_score(df3_clean['Gender'],df2['Gender'],average='micro')
print("Error of Gender :", math.trunc((1 - cm)*100),'%')

cm = recall_score(df3_clean['Diabetes Medication'],df2['Diabetes Medication'],average='micro')
print("Error of Diabetes Medication : ", math.trunc((1 - cm)*100),'%')

num_lab_procedures_error = mean_absolute_percentage_error(df2['Num Lab Procedures'], df3_clean['Num Lab Procedures'])
print('Error of NumLabProcedures: ', math.trunc(num_lab_procedures_error*100),'%')

time_in_hospital_error = mean_absolute_percentage_error(df2['Time In Hospital'], df3_clean['Time In Hospital'])
print('Error of TimeInHospital: ', math.trunc(time_in_hospital_error*100),'%')

As we can notice error is low, from 0.2% to 5.3%. So we can rely on this type of data recovering.

<h3 id="correct_data_format">Correct data format</h3>
<b>We are almost there!</b>
<p>The last step in data cleaning is checking and making sure that all data is in the correct format (int, float, text or other).</p>

In Pandas, we use:

<p><b>.dtype()</b> to check the data type</p>
<p><b>.astype()</b> to change the data type</p>


<h4>Let's list the data types for each column</h4>


In [ ]:
df.dtypes

<p>As we can see above, some columns are not of the correct data type. Numerical variables should have type 'float' or 'int', and variables with strings should have type 'object'. For example, 'Gender', 'Age' variables are Object values that descraibes categorical data, so we should expect them to be 'category' type. We have to convert data types into a proper format for each column using the "astype()" method.</p>


<h4>Convert data types to proper format</h4>


In [ ]:
df[["Gender", "Age"]] = df[["Gender", "Age"]].astype("category")

<h4>Let us list the columns after the conversion</h4>


In [ ]:
df.dtypes

<b>Wonderful!</b>

Now we have finally obtained the cleaned dataset with no missing values with all data in its proper format.


<h2 id="data_standardization">Data Standardization</h2>
<p>
Data is usually collected from different agencies in different formats.
(Data standardization is also a term for a particular type of data normalization where we subtract the mean and divide by the standard deviation.)
</p>

<b>What is standardization?</b>

<p>Standardization is the process of transforming data into a common format, allowing the researcher to make the meaningful comparison.
</p>

<b>Example</b>

<p>Transform Diabetes Medication to Prescribed Diabetes Medication:</p>
<p>We will need to apply <b>data transformation</b> to transform Diabetes Medication into Prescribed Diabetes Medication.</p>



<p>The code for unit conversion is:<p>
df["Diabetes Medication"] = 'Yes' == df["Diabetes Medication"]
<p>If we need to, we can do many mathematical operations directly in Pandas.</p>


In [ ]:
df['Diabetes Medication'].head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #2: </h1>

<b>According to the example above, transform "Yes" "No" to True False in the column of "Diabetes Medication" and change the name of column to "Prescribed Diabetes Medication".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
# Convert Yes, No values to Boolean True, False
df['Diabetes Medication'] = 'Yes' == df["Diabetes Medication"]
# rename column name from "Diabetes Medication" to "Prescribed Diabetes Medication"

df.rename(columns={'Diabetes Medication':'Prescribed Diabetes Medication'}, inplace=True)

# check your transformed data
df['Prescribed Diabetes Medication'].head()

```

</details>


<h2 id="data_normalization">Data Normalization</h2>

<b>Why normalization?</b>

<p>Normalization is the process of transforming values of several variables into a similar range. Typical normalizations include scaling the variable so the variable average is 0, scaling the variable so the variance is 1, or scaling the variable so the variable values range from 0 to 1.
</p>

<b>Example</b>

<p>To demonstrate normalization, let's say we want to scale the columns "NumberDiagnoses" and "NumLabProcedures".</p>
<p><b>Target:</b> would like to normalize those variables so their value ranges from 0 to 1</p>
<p><b>Approach:</b> replace original value by (original value)/(maximum value)</p>


In [ ]:
df['Number Diagnoses'] = df['Number Diagnoses']/df['Number Diagnoses'].max()

<h2>Testing for normality using statistical tests</h2>

Here we will check the normality of the sample distribution using such tests as: the Shapiro–Wilk test, the Kolmogorov–Smirnov test, and the D'Agostino's test.

In [ ]:
values = df["Number Diagnoses"]

results = {
    "Shapiro–Wilk test" : stats.shapiro(values),
    "Kolmogorov–Smirnov" : stats.kstest(values, "norm"),
    "D'Agostino's test" : stats.normaltest(values)
}

pd.DataFrame.from_dict(results, orient = "index", columns = ["Test statistics", "Value p"])

According to the results of the tests, it can be reliably stated that this sample does not have a normal distribution law.

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #3: </h1>

<b>According to the example above, normalize the column "NumLabProcedures".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
df['Num Lab Procedures'] = df['Num Lab Procedures']/df['Num Lab Procedures'].max()

# show the scaled columns
df[["Number Diagnoses","Num Lab Procedures"]].head()

```

</details>

Here we can see we've normalized "NumberDiagnoses" and "NumLabProcedures" in the range of \[0,1].


<h2 id="binning">Binning</h2>
<b>Why binning?</b>
<p>
    Binning is a process of transforming continuous numerical variables into discrete categorical 'bins' for grouped analysis.
</p>

<b>Example: </b>

<p>In our dataset, "Age" is a real valued variable ranging from 0 to 100 and it has 10 unique values. What if we only care about the diabetes among 4 age groups child, young adults, middle-aged adults and old-aged adults? Can we rearrange them into four ‘bins' to simplify analysis? </p>

<p>We will use the pandas method 'cut' to segment the 'Age' column into 4 bins.</p>


<h3>Example of Binning Data In Pandas</h3>


Convert data to correct format:


In [ ]:
df["Age"]=df["Age"].astype('category', copy=True)

Let's plot the histogram of age to see what the distribution of age looks like.


In [ ]:
plt.hist(df["Age"])

# set x/y labels and plot title
plt.xlabel("age")
plt.ylabel("count")
plt.title("age distribution")

<p>Since we are building 4 bins, there should be 5 dividers, so numbers_generated = 5.</p>


We build a bin array with a minimum value to a maximum value by using the bandwidth calculated above. The values will determine when one bin ends and another begins.



We set group  names:


In [ ]:
group_names = ['Child', 'Young Adults', 'Middle-Aged Adults',
            'Old-Aged Adults']

We apply the function "cut" to determine what each value of `df['Age']` belongs to.


In [ ]:
df['ages-binned'] = pd.cut(df['Age'].astype('string').str[1:2].astype('int64'), bins=[0,1,2,4,10], labels=group_names, include_lowest=True)
df[['Age','ages-binned']].head(20)

This code fragment is used, because "Age" is a category. So that we will select 1 element of string from each occurrence. As a result we are able to represent category as 1d array for function to proceed.

Let's see the number of patients in each bin:

In [ ]:
df["ages-binned"].value_counts()

Let's plot the distribution of each bin:

In [ ]:
pyplot.bar(['Old-Aged', 'Middle-Aged', 'Young','Child'], df["ages-binned"].value_counts())

# set x/y labels and plot title
plt.xlabel("age groups")
plt.ylabel("count")
plt.title("age groups bins")

<p>
    Look at the dataframe above carefully. You will find that the last column provides the bins for "Age" based on 4 categories ("Old-Aged", "Middle-Aged", "Young","Child").
</p>
<p>
    We successfully narrowed down the intervals from 10 to 4!
</p>


<h3>Bins Visualization</h3>
Normally, a histogram is used to visualize the distribution of bins we created above. 


In [ ]:
# draw historgram of attribute "age" with 4 selected bins (0,1,2,4,10)
plt.hist(df["Age"], bins = [0,1,2,4,10])

# set x/y labels and plot title
plt.xlabel("age")
plt.ylabel("count")
plt.title("age groups bins")

The plot above shows the binning result for the attribute "Age".


<h2 id="indicator">Indicator Variable (or Dummy Variable)</h2>
<b>What is an indicator variable?</b>
<p>
    An indicator variable (or dummy variable) is a numerical variable used to label categories. They are called 'dummies' because the numbers themselves don't have inherent meaning. 
</p>

<b>Why we use indicator variables?</b>

<p>
    We use indicator variables so we can use categorical variables for regression analysis in the later modules.
</p>
<b>Example</b>
<p>
    We see the column "Change" has two unique values: "Ch" or "No". Regression doesn't understand words, only numbers. To use this attribute in regression analysis, we convert "change" to indicator variables.
</p>

<p>
    We will use pandas' method 'get_dummies' to assign numerical values to different categories of change type.
</p>


In [ ]:
df.columns

Get the indicator variables and assign it to data frame "dummy_variable_1":


In [ ]:
dummy_variable_1 = pd.get_dummies(df["Change"])
dummy_variable_1.head()

Change the column names for clarity:


In [ ]:
dummy_variable_1.rename(columns={'Ch':'change_yes', 'No':'change_no'}, inplace=True)
dummy_variable_1.head()

In the dataframe, column 'change' has values for 'yes' and 'no' as 0s and 1s now.


In [ ]:
# merge data frame "df" and "dummy_variable_1" 
df = pd.concat([df, dummy_variable_1], axis=1)

# drop original column "change" from "df"
df.drop("Change", axis = 1, inplace=True)

In [ ]:
df.head()

The last two columns are now the indicator variable representation of the change variable. They're all 0s and 1s now.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4: </h1>

<b>Similar to before, create an indicator variable for the column "Rosiglitazone"</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
# get indicator variables of aspiration and assign it to data frame "dummy_variable_2"
dummy_variable_2 = pd.get_dummies(df['Rosiglitazone'])

# change column names for clarity
dummy_variable_2.rename(columns={'Down':'Increased', 'Up': 'Decreased'}, inplace=True)

# show first 5 instances of data frame "dummy_variable_1"
dummy_variable_2.head()

```
</details>


 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #5: </h1>

<b>Merge the new dataframe to the original dataframe, then drop the column 'Rosiglitazone'.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute

<details><summary>Click here for the solution</summary>

```python
# merge the new dataframe to the original datafram
df = pd.concat([df, dummy_variable_2], axis=1)

# drop original column "Rosiglitazone" from "df"
df.drop('Rosiglitazone', axis = 1, inplace=True)

```
</details>


In [ ]:
df.to_csv('clean_df.csv')

Save the new csv:

> Note : The  csv file cannot be viewed in the jupyterlite based SN labs environment.


### Thank you for completing this lab!

## Author

<a href="https://author.skills.network/instructors/dmytro_yesyp" target="_blank">Dmytro Yesyp</a>

### Other Contributors

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/nataliya_boyko">Ass. Prof. Nataliya Boyko, PhD</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                         |
| ----------------- | ------- | ---------- | ---------------------------------------------------------- |
| 2023-03-04       | 01| Dmytro Yesyp     | Lab created|
<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>
